In [1]:
import torch
from transformers import BertForTokenClassification, BertTokenizer
from termcolor import colored
import copy
import stanza
try:
    nlp = stanza.Pipeline(lang='en', processors='tokenize')
except Exception:
    stanza.download('en')
    nlp = stanza.Pipeline(lang='en', processors='tokenize')

2022-05-03 02:39:19 INFO: Loading these models for language: en (English):
| Processor | Package  |
------------------------
| tokenize  | combined |

2022-05-03 02:39:19 INFO: Use device: gpu
2022-05-03 02:39:19 INFO: Loading: tokenize


2022-05-03 02:39:21 INFO: Done loading processors!


In [2]:
text = '''
Admission Date:  [**2118-6-2**]       Discharge Date:  [**2118-6-14**]  Date of Birth:                    Sex:  F  Service:  MICU and then to [**Doctor Last Name **] Medicine  HISTORY OF PRESENT ILLNESS:  This is an 81-year-old female with a history of emphysema (not on home O2), who presents with three days of shortness of breath thought by her primary care doctor to be a COPD flare.  Two days prior to admission, she was started on a prednisone taper and one day prior to admission she required oxygen at home in order to maintain oxygen saturation greater than 90%.  She has also been on levofloxacin and nebulizers, and was not getting better, and presented to the [**Hospital1 18**] Emergency Room.  In the [**Hospital3 **] Emergency Room, her oxygen saturation was 100% on CPAP.  She was not able to be weaned off of this despite nebulizer treatment and Solu-Medrol 125 mg IV x2.  Review of systems is negative for the following:  Fevers, chills, nausea, vomiting, night sweats, change in weight, gastrointestinal complaints, neurologic changes, rashes, palpitations, orthopnea.  Is positive for the following: Chest pressure occasionally with shortness of breath with exertion, some shortness of breath that is positionally related, but is improved with nebulizer treatment.  PAST MEDICAL HISTORY: 1. COPD.  Last pulmonary function tests in [**2117-11-3**] demonstrated a FVC of 52% of predicted, a FEV1 of 54% of predicted, a MMF of 23% of predicted, and a FEV1:FVC ratio of 67% of predicted, that does not improve with bronchodilator treatment.  The FVC, however, does significantly improve with bronchodilator treatment consistent with her known reversible air flow obstruction in addition to an underlying restrictive ventilatory defect.  The patient has never been on home oxygen prior to this recent episode.  She has never been on steroid taper or been intubated in the past. 2. Lacunar CVA.  MRI of the head in [**2114-11-4**] demonstrates "mild degree of multiple small foci of high T2 signal within the white matter of both cerebral hemispheres as well as the pons, in the latter region predominantly to the right of midline.  The abnormalities, while nonspecific in etiology, are most likely secondary to chronic microvascular infarction.  There is no mass, lesion, shift of the normal midline strictures or hydrocephalus.  The major vascular flow patterns are preserved.  There is moderate right maxillary, moderate bilateral ethmoid, mild left maxillary, minimal right sphenoid, and frontal sinus mucosal thickening.  These abnormalities could represent an allergic or some other type of inflammatory process.  Additionally noted is a moderately enlarged subtotally empty sella turcica". 3. Angina:  Most recent stress test was in [**2118-1-3**] going for four minutes with a rate pressure product of 10,000, 64% of maximum predicted heart rate without evidence of ischemic EKG changes or symptoms.  The imaging portion of the study demonstrated no evidence of myocardial ischemia and a calculated ejection fraction of 84%.  The patient denies angina at rest and gets angina with walking a few blocks. Are alleviated by sublingual nitroglycerin. 4. Hypothyroidism on Synthroid. 5. Depression on Lexapro. 6. Motor vehicle accident with head injury approximately 10 years ago.  MEDICATIONS ON ADMISSION: 1. Hydrochlorothiazide 25 q.d. 2. Prednisone 60 mg, 50 mg, 40 mg, 20 mg. 3. Levofloxacin 500 mg q.d. 4. Imdur 60 mg q.d. 5. Synthroid 75 mcg q.d. 6. Pulmicort nebulizer b.i.d. 7. Albuterol nebulizer q.4. prn. 8. Lexapro 10 mg q.d. 9. Protonix 40 mg q.d. 10. Aspirin 81 mg q.d.  ALLERGIES:  Norvasc leads to lightheadedness and headache.  FAMILY HISTORY:  Noncontributory.  SOCIAL HISTORY:  Lives with her husband, Dr. [**Known lastname 1809**] an eminent Pediatric Neurologist at [**Hospital3 1810**].  The patient is a prior smoker, but has not smoked in over 10 years.  She has no known alcohol use and she is a full code.  PHYSICAL EXAM AT TIME OF ADMISSION:  Blood pressure 142/76, heart rate 100 and regular, respirations at 17-21, and 97% axillary temperature.  She was saturating at 100% on CPAP with dry mucous membranes.  An elderly female in no apparent distress.  Pupils are equal, round, and reactive to light and accommodation.  Extraocular movements are intact.  Oropharynx difficult to assess due to CPAP machine.  No evidence of jugular venous pressure, however, the strap from the CPAP machine obscures the neck exam.  Cranial nerves II through XII are grossly intact.  Neck is supple without lymphadenopathy.  Heart exam:  Tachycardic, regular, obscured by loud bilateral wheezing with increase in the expiratory phase as well as profuse scattered rhonchi throughout the lung fields.  Positive bowel sounds, soft, nontender, nondistended, obese, no masses.  Mild edema of the lower extremities without clubbing or cyanosis, no rashes.  There is a right hand hematoma.  Strength is assessed as [**5-9**] in the lower extremities, [**5-9**] in the upper extremities with a normal mental status and cognition.  LABORATORY STUDIES:  White count 19, hematocrit 41, platelets 300.  Chem-7:  127, 3.6, 88, 29, 17, 0.6, 143.  Troponin was negative.  CKs were negative times three.  Initial blood gas showed a pH of 7.4, pO2 of 66, pCO2 of 54.  Chest x-ray demonstrates a moderate sized hiatal hernia, segmental atelectasis, left lower lobe infiltrate versus segmental atelectasis.  EKG shows normal sinus rhythm at 113 beats per minute, normal axis, no evidence of ST-T wave changes.  BRIEF SUMMARY OF HOSPITAL COURSE: 1. COPD/dyspnea/pneumonia:  The patient was initially placed on an aggressive steroid taper and admitted to the Medical Intensive Care Unit due to her difficulty with oxygenation despite CPAP machine.  She was also given nebulizer treatments q.4h. as well as chest PT.  The nebulizers were increased to q.1h. due to the fact that she continued to have labored breathing.  Due to persistent respiratory failure and labored breathing, the patient was intubated on [**2118-6-7**] in order to improve oxygenation, ventilation, and ability to suction.  A bronchoscopy was performed on [**2118-6-7**], which demonstrated marked narrowing of the airways with expiration consistent with tracheomalacia.  On [**2118-6-9**], two silicone stents were placed, one in the left main stem (12 x 25 and one in the trachea 16 x 40) by Dr. [**First Name (STitle) **] [**Name (STitle) **] under rigid bronchoscopy with general anesthesia.  On [**2118-6-11**], the patient was extubated to a cool mist shovel mask and her oxygen was titrated down to 2 liters nasal cannula at which time she was transferred to the medical floor.  On the medical floor, the steroids were weaned to off on [**2118-6-14**], and the patient was saturating at 97% on 2 liters, 92% on room air.  On [**2118-6-14**], the patient was seen again by the Interventional Pulmonology service, who agreed that she looked much improved and recommended that she go to pulmonary rehabilitation with followup within six weeks\cf4 \\'\cf2  time status post placement of stents in respiratory failure.  2. Cardiovascular:  The patient was ruled out for a MI.  She did have another episode on the medical floor of chest pain, which showed no evidence of EKG changes and negative troponin, negative CKs x3.  She was continued on aspirin, Imdur, and diltiazem for rate control per her outpatient regimen.  3. Hypertension:  She was maintained on diltiazem and hydrochlorothiazide with adequate blood pressure control and normalization of electrolytes.  4. Hematuria:  The patient had intermittent hematuria likely secondary to Foley placement.  The Foley catheter was discontinued on [**2118-6-14**].  She had serial urinalyses, which were all negative for signs of infection.  5. Hyperglycemia:  Patient was placed on insulin-sliding scale due to hyperglycemia, which was steroid induced.  This worked quite well and her glucose came back to normal levels once the steroids were tapered to off.  6. Leukocytosis:  Patient did have a profound leukocytosis of 20 to 22 during much of her hospital course.  As the steroids were tapered to off, her white blood cell count on [**2118-6-14**] was 15,000.  It was felt that the leukocytosis was secondary to both steroids as well as question of a left lower lobe pneumonia.  7. For the left lower lobe pneumonia, the patient had initially received a course of levofloxacin 500 p.o. q.d. from [**2118-6-4**] to [**2118-6-10**].  This was restarted on [**2118-6-12**] for an additional seven day course given the fact that she still had the leukocytosis and still had marked rales at the left lower lobe.  8. Hypothyroidism:  The patient was continued on outpatient medical regimen.  9. Depression:  The patient was continued on Lexapro per outpatient regimen.  It is recommended that she follow up with a therapist as an outpatient due to the fact that she did have a blunted affect throughout much of the hospital course, and did appear clinically to be depressed.  10. Prophylaxis:  She was maintained on proton-pump inhibitor with subQ Heparin.  11. Sore throat:  The patient did have a sore throat for much of the hospital course post extubation.  This was treated with Cepacol lozenges as well as KBL liquid (a solution containing Kaopectate, Bismuth, and lidocaine) at bedtime.  12. Communication/code status:  The patient was full code throughout her hospital course, and communication was maintained with the patient and her husband.  13. Muscle weakness:  The patient did have profound muscle weakness and was evaluated by Physical Therapy, and was found to have impaired functional mobility, impaired musculoskeletal performance, impaired gas exchange, impaired endurance, impaired ventilation, and needed help with supine to sit.  However, she was able to tolerate sitting in a chair for approximately one hour.  On motor exam, her flexors and extensors of the lower extremities were [**4-8**] at the knee, [**4-8**] at the ankle, [**4-8**] at the elbows, and [**4-8**] hips.  It was felt that this weakness was most likely due to a combination of steroid myopathy as well as muscle atrophy secondary to deconditioning after a prolonged hospital course.  14. Speech/swallow:  The patient had a Speech and Swallow evaluation showing no evidence of dysphagia, no evidence of vocal cord damage status post tracheal stent placement.  DISCHARGE CONDITION:  The patient was able to oxygenate on room air at 93% at the time of discharge.  She was profoundly weak, but was no longer tachycardic and had a normal blood pressure.  Her respirations were much improved albeit with transmitted upper airway sounds.  DISCHARGE STATUS:  The patient will be discharged to [**Hospital1 **] for both pulmonary and physical rehabilitation.  DISCHARGE MEDICATIONS: 1. Levothyroxine 75 mcg p.o. q.d. 2. Citalopram 10 mg p.o. q.d. 3. Aspirin 81 mg p.o. q.d. 4. Fluticasone 110 mcg two puffs inhaled b.i.d. 5. Salmeterol Diskus one inhalation b.i.d. 6. Acetaminophen 325-650 mg p.o. q.4-6h. prn. 7. Ipratropium bromide MDI two puffs inhaled q.2h. prn. 8. Albuterol 1-2 puffs inhaled q.2h. prn. 9. Zolpidem tartrate 5 mg p.o. q.h.s. prn. 10. Isosorbide dinitrate 10 mg p.o. t.i.d. 11. Diltiazem 60 mg p.o. q.i.d. 12. Pantoprazole 40 mg p.o. q.24h. 13. Trazodone 25 mg p.o. q.h.s. prn. 14. SubQ Heparin 5000 units subcutaneous b.i.d. until such time that the patient is able to get out of bed twice a day. 15. Cepacol lozenges q.2h. prn. 16. Levofloxacin 500 mg p.o. q.d. for a seven day course to be completed on [**2118-6-21**]. 17. Kaopectate/Benadryl/lidocaine 5 mL p.o. b.i.d. prn, not to be given around mealtimes for concern of dysphagia induced by lidocaine. 18. Lorazepam 0.5-2 mg IV q.6h. prn.  FOLLOW-UP PLANS:  The patient is recommended to followup with Dr. [**First Name4 (NamePattern1) **] [**Last Name (NamePattern1) 1407**], [**Telephone/Fax (1) 1408**] within two weeks of leaving of the hospital.  She is also recommended to followup with the Interventional Pulmonary service for followup status post stent placement.  She is also recommended to followup with a neurologist if her muscle weakness does not improve within one week on physical therapy with concern for steroid-induced myopathy.  FINAL DIAGNOSES: 1. Tracheomalacia status post tracheal and left main stem bronchial stent placement. 2. Hypertension. 3. Hypothyroidism. 4. Restrictive lung defect. 5. Depression.                        DR.[**Last Name (STitle) **],[**First Name3 (LF) **] 12-207   Dictated By:[**Last Name (NamePattern1) 1811**] MEDQUIST36  D:  [**2118-6-14**]  11:30 T:  [**2118-6-14**]  11:33 JOB#:  [**Job Number 1812**]'''

In [3]:
import sys
sys.path.append('../')
from NER.processor import *
from NER.ner_utils import *
from NER import config

Using  cuda


In [4]:
num_labels = len(tag2idx)
save_model_address = '../trained_models/NER/C-Bert-test'
model = BertForTokenClassification.from_pretrained(save_model_address, num_labels=num_labels)
tokenizer = BertTokenizer.from_pretrained(save_model_address, do_lower_case=False)


In [5]:
def predict_long_text(long_text):
    all_sentences = []
    all_tags = []
    doc = nlp(long_text)
    for i, sentence in enumerate(doc.sentences):
        # temp_token: tokenized words
        # input_ids: convert temp_token to id
        temp_token, input_ids, attention_masks = create_query(sentence, tokenizer)
        result_list = model_inference(model, input_ids)
        result = [tag2name[t] for t in result_list]
        pretok_sent = ""
        pretags = ""
        for i, tok in enumerate(temp_token):
            if tok.startswith("##"):
                pretok_sent += tok[2:]
            else:
                pretok_sent += f" {tok}"
                pretags += f" {result[i]}"
        pretok_sent = pretok_sent[1:]
        pretags = pretags[1:]
        s = pretok_sent.split()
        t = pretags.split()
        all_sentences.append(s)
        all_tags.append(t)
    return all_sentences, all_tags

In [6]:
all_sentences, all_tags = predict_long_text(text)

In [7]:
sentences_with_problem = []
all_problems_in_text_tmp = []
all_treatment_in_text = []
all_test_in_text = []

for s, t in zip(all_sentences, all_tags):
    flag_treatment, flag_problem, flag_test = 0, 0, 0
    problem_in_sentence = ''
    treatment_in_sentence = []
    test_in_sentence = []

    for i in range(1, len(t)-1):
        if t[i] == 'B-problem':
            flag_problem = 1
            # if there is entities, add the index of sentence to a list
            # sentences_with_problem.append(n)
            # append the index of entity to a list
            if problem_in_sentence: 
                problem_in_sentence = f'{problem_in_sentence}| {str(i)}'
            else: 
                problem_in_sentence += str(i)
        elif t[i] == 'I-problem' or t[i] == 'X' and flag_problem == 1:
            problem_in_sentence = f'{problem_in_sentence} {str(i)}'

        elif t[i] == 'B-test':
            flag_test = 1
            test_in_sentence.append(i)
        elif t[i] == 'I-test' or t[i] == 'X' and flag_test == 1 :
            test_in_sentence.append(i)

        elif t[i] == 'B-treatment':
            flag_treatment = 1
            treatment_in_sentence.append(i)
        elif t[i] == 'I-treatment' or t[i] == 'X' and flag_treatment == 1 :
            treatment_in_sentence.append(i)   

        elif t[i] in ['O', 'X']:
            flag_treatment, flag_problem, flag_test = 0, 0, 0 
            # print(s[i], end=' ')

    all_problems_in_text_tmp.append(problem_in_sentence)
    all_treatment_in_text.append(treatment_in_sentence)
    all_test_in_text.append(test_in_sentence)

In [8]:
# get index of entities in each sentence
all_problems_in_text = []
for x in all_problems_in_text_tmp:
    if x:
        index = x.split('|')
        tmp = [i.split() for i in index]
        all_problems_in_text.append(tmp)
    else: 
        all_problems_in_text.append(x)
        


In [13]:
# create sentences with '[entity]' tag
sentences_with_problem = []
for sentence, problem_index in zip(all_sentences, all_problems_in_text):
    # print(problem_index)
    if problem_index:
        for i_list in problem_index:
            s = copy.deepcopy(sentence)
            s.insert(int(i_list[-1])+1, '[entity]')
            s.insert(int(i_list[0]), '[entity]')
            s = ' '.join(s)
            sentences_with_problem.append(s)
        
        

In [14]:
sentences_with_problem

['[CLS] This is an 81 - year - old female with a history of [entity] emphysema [entity] ( not on home O2 ) , who presents with three days of shortness of breath thought by her primary care doctor to be a COPD flare . [SEP]',
 '[CLS] This is an 81 - year - old female with a history of emphysema ( not on home O2 ) , who presents with three days of [entity] shortness of breath [entity] thought by her primary care doctor to be a COPD flare . [SEP]',
 '[CLS] This is an 81 - year - old female with a history of emphysema ( not on home O2 ) , who presents with three days of shortness of breath thought by her primary care doctor to be [entity] a COPD flare [entity] . [SEP]',
 '[CLS] Review of systems is negative for the following : [entity] Fevers [entity] , chills , nausea , vomiting , night sweats , change in weight , gastrointestinal complaints , neurologic changes , rashes , palpitations , orthopnea . [SEP]',
 '[CLS] Review of systems is negative for the following : Fevers , [entity] chills

In [16]:
from transformers import AutoConfig, AutoTokenizer, BertForSequenceClassification
# no of classifier: present, not-present
num_labels = 3
MODEL_CLASSES = {
  'bert': (AutoConfig, BertForSequenceClassification, AutoTokenizer),
}
MODEL_ADDRESS = 'emilyalsentzer/Bio_ClinicalBERT'
config_class, model_class, tokenizer_class = MODEL_CLASSES['bert']
model_config = config_class.from_pretrained(MODEL_ADDRESS, num_labels=num_labels)
tokenizer = tokenizer_class.from_pretrained(MODEL_ADDRESS, do_lower_case=False)
model = model_class.from_pretrained(MODEL_ADDRESS, config=model_config)
output_dir = '../trained_models/Assertion/3_lable_model'
model = model_class.from_pretrained(output_dir)
tokenizer = tokenizer_class.from_pretrained(output_dir)

Some weights of the model checkpoint at emilyalsentzer/Bio_ClinicalBERT were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model

In [17]:
input_ids = []
attention_masks = []
# For every sentence...
for sent in sentences_with_problem:
    encoded_dict = tokenizer.encode_plus(
                        sent,                      # Sentence to encode.
                        add_special_tokens = False, # Add '[CLS]' and '[SEP]'
                        max_length = 128,           # Pad & truncate all sentences.
                        pad_to_max_length = True,
                        return_attention_mask = True,   # Construct attn. masks.
                        return_tensors = 'pt',     # Return pytorch tensors.
                   )

    input_ids.append(encoded_dict['input_ids'])
    attention_masks.append(encoded_dict['attention_mask'])

# Convert the lists into tensors.
input_ids = torch.cat(input_ids, dim=0)
attention_masks = torch.cat(attention_masks, dim=0)
# labels = torch.tensor(labels)

Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:2271: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


In [19]:
input_ids = torch.tensor(input_ids)
attention_masks = torch.tensor(attention_masks)

<ipython-input-19-1be4e8e87b42>:1: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  input_ids = torch.tensor(input_ids)
<ipython-input-19-1be4e8e87b42>:2: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  attention_masks = torch.tensor(attention_masks)


In [20]:
predictions , true_labels = [], []

In [ ]:
model.eval()

with torch.no_grad():
    result = model(input_ids, token_type_ids=None, attention_mask=attention_masks, return_dict=True)

logits = result.logits
logits = logits.detach().cpu().numpy()
predictions.append(logits)

# print('sentences: ', sentences)
pred_labels_i = np.argmax(logits, axis=1).flatten()
# print('Label prediction: ', pred_labels_i) 

for index, sentence in enumerate(sentences_with_problem):
  print(sentence)
  print(pred_labels_i[index])
  if pred_labels_i[index] == 0:
    print ('Present')
  elif pred_labels_i[index] == 1:
    print ('Possible')
  elif pred_labels_i[index] == 2:
    print ('Not-present')


In [22]:
pred_labels_i

array([0, 0, 0, 0, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 0,
       0, 2, 2, 2, 2, 0, 1, 0, 2, 2, 2, 2, 2, 0, 0, 0, 0, 0, 0, 2, 2, 2,
       0, 0, 0, 2, 2, 2, 2, 0, 2, 2, 2, 0, 0, 0, 1, 1, 2, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 2, 0, 2, 0, 0, 0, 2, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 2, 0, 2, 0, 1,
       0, 1, 0, 0, 0, 0, 0])

In [18]:
clinical_text = [['He',
 'was',
 'admitted',
 ',',
 'taken',
 'to',
 'the',
 'operating',
 'room',
 'where',
 'he',
 'underwent',
 'L5-S1',
 'right',
 'hemilaminectomy',
 'and',
 'discectomy',
 '.']
, 
['Over',
 'the',
 'next',
 'three',
 'days',
 'he',
 'increased',
 'his',
 'activity',
 'gradually',
 ',',
 'was',
 'able',
 'to',
 'do',
 'stairs',
 'with',
 'Physical',
 'Therapy',
 'and',
 'had',
 'pain',
 'which',
 'could',
 'be',
 'controlled',
 'with',
 'oral',
 'analgesics',
 '.'],

['However', ',', 'her', 'creatinine', 'continued', 'to', 'increase', '.']
]

In [26]:
problem_list = [[[4, 5, 6], [10]], '', [[2, 3], [6,7]]]

In [24]:
labels = [['yes', 'no'], '', ['yes', 'yes']]

In [33]:
ss = clinical_text[0]
pp = problem_list[1]
ll = labels[0]
index_in_sentence = []
for i, index in enumerate(pp):
    n = len(index)
    index_in_sentence.append([ll[i]] * n)
print(index_in_sentence)
    
    

[]


In [44]:
r = map(lambda l: [item for elem in l for item in elem], problem_list)
print(list(r))

In [34]:

flatList = [ item for elem in pp for item in elem]


In [35]:
flatList

[]

In [25]:
for sentence, problem_index in zip(clinical_text, problem_list):
    # print(sentence)
    if problem_index:
        for i_list in problem_index:
            s = copy.deepcopy(sentence)
            print(i_list)
            print("=====")
            # print(i_list)
            s.insert(int(i_list[-1])+1, '[entity]')
            s.insert(int(i_list[0]), '[entity]')
            s = ' '.join(s)
            print(s)


[4, 5]
=====
He was admitted , [entity] taken to [entity] the operating room where he underwent L5-S1 right hemilaminectomy and discectomy .
[8]
=====
He was admitted , taken to the operating [entity] room [entity] where he underwent L5-S1 right hemilaminectomy and discectomy .
[2, 3]
=====
However , [entity] her creatinine [entity] continued to increase .
[6, 7]
=====
However , her creatinine continued to [entity] increase . [entity]


In [123]:
clinical_text

[['He',
  'was',
  'admitted',
  ',',
  '[entity]',
  'taken',
  'to',
  '[entity]',
  '[entity]',
  'the',
  '[entity]',
  'operating',
  'room',
  'where',
  'he',
  'underwent',
  'L5-S1',
  'right',
  'hemilaminectomy',
  'and',
  'discectomy',
  '.'],
 ['Over',
  'the',
  'next',
  'three',
  'days',
  'he',
  'increased',
  'his',
  'activity',
  'gradually',
  ',',
  'was',
  'able',
  'to',
  'do',
  'stairs',
  'with',
  'Physical',
  'Therapy',
  'and',
  'had',
  'pain',
  'which',
  'could',
  'be',
  'controlled',
  'with',
  'oral',
  'analgesics',
  '.'],
 ['However', ',', 'her', 'creatinine', 'continued', 'to', 'increase', '.']]

In [6]:
clinical_tags = [['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-treatment',
 'I-treatment',
 'I-treatment',
 'O',
 'B-treatment',
 'O'],
                 
 ['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-treatment',
 'I-treatment',
 'O',
 'O',
 'B-problem',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-treatment',
 'I-treatment',
 'O'],
 
 ['O', 'O', 'B-test', 'I-test', 'O', 'O', 'O', 'O']                
]


In [51]:
query_input_ids, query_input_tags, query_attention_masks = process_data(clinical_text, clinical_tags, tokenizer)

In [52]:
query_inputs = torch.tensor(query_input_ids)
query_tags = torch.tensor(query_input_tags)
query_masks = torch.tensor(query_attention_masks)


In [53]:
query_masks.shape

torch.Size([3, 64])

In [54]:
def model_evaluation(input_ids, label_ids, input_mask):
    y_true = []
    y_pred = []
    model.eval()
    with torch.no_grad():
        outputs = model(input_ids, token_type_ids=None,
        attention_mask=input_mask)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 
    # Get NER predict result
    logits = torch.argmax(F.log_softmax(logits, dim=2), dim=2)
    logits = logits.detach().cpu().numpy()
    # Get NER true result
    label_ids = label_ids.to('cpu').numpy()
    # Only predict the real word, mark=0, will not calculate
    input_mask = input_mask.to('cpu').numpy()
    # Compare the valuable predict result
    for i, mask in enumerate(input_mask):
        # Real one
        temp_1 = []
        # Predict one
        temp_2 = []
        for j, m in enumerate(mask):
            # Mark=0, meaning its a pad word, dont compare
            if m:
                if tag2name[label_ids[i][j]] != "X" and tag2name[label_ids[i][j]] != "[CLS]" and tag2name[label_ids[i][j]] != "[SEP]" : # Exclude the X label
                    # print(tag2name[logits[i][j]])
                    temp_1.append(tag2name[label_ids[i][j]])
                    temp_2.append(tag2name[logits[i][j]])
            else:
                break
        y_true.append(temp_1)
        y_pred.append(temp_2)

    return y_true, y_pred


In [55]:
y_true, y_pred = model_evaluation(query_inputs, query_tags, query_masks)
    

In [57]:
report = classification_report(y_true, y_pred, digits=4)


In [60]:
# Get acc , recall, F1 result report
report = classification_report(y_true, y_pred, digits=4)

# Save the report into file
output_eval_file = "eval_results.txt"

with open(output_eval_file, "w") as writer:
    print("***** Eval results *****")
    print("\n%s"%(report))
    print("f1 socre: %f"%(f1_score(y_true, y_pred)))
    print("Accuracy score: %f"%(accuracy_score(y_true, y_pred)))
    
    writer.write("f1 socre:\n")
    writer.write(str(f1_score(y_true, y_pred)))
    writer.write("\n\nAccuracy score:\n")
    writer.write(str(accuracy_score(y_true, y_pred)))
    writer.write("\n\n")  
    writer.write(report)

***** Eval results *****

              precision    recall  f1-score   support

     problem     1.0000    1.0000    1.0000         1
        test     1.0000    1.0000    1.0000         1
   treatment     1.0000    1.0000    1.0000         4

   micro avg     1.0000    1.0000    1.0000         6
   macro avg     1.0000    1.0000    1.0000         6
weighted avg     1.0000    1.0000    1.0000         6

f1 socre: 1.000000
Accuracy score: 1.000000


In [63]:
report.split('\n')

['              precision    recall  f1-score   support',
 '',
 '     problem     1.0000    1.0000    1.0000         1',
 '        test     1.0000    1.0000    1.0000         1',
 '   treatment     1.0000    1.0000    1.0000         4',
 '',
 '   micro avg     1.0000    1.0000    1.0000         6',
 '   macro avg     1.0000    1.0000    1.0000         6',
 'weighted avg     1.0000    1.0000    1.0000         6',
 '']

In [9]:
model = BertForTokenClassification.from_pretrained(bert_out_address, num_labels=len(tag2idx))
tokenizer = BertTokenizer.from_pretrained(bert_out_address, do_lower_case=False)


In [13]:
test_query = ' '.join(test_query)


In [14]:
test_query

'Over the next three days he increased his activity gradually , was able to do stairs with Physical Therapy and had pain which could be controlled with oral analgesics .'

In [15]:
tokenized_texts = []
temp_token = []
temp_token.append('[CLS]')
token_list = tokenizer.tokenize(test_query)
temp_token.extend(token_list)
temp_token = temp_token[:max_len-1]
temp_token.append('[SEP]')
input_id = tokenizer.convert_tokens_to_ids(temp_token)
padding_len = max_len - len(input_id)
input_id = input_id + ([0] * padding_len)
tokenized_texts = []
tokenized_texts.append(input_id)
attention_masks = [[int(i>0) for i in input_id]]

tokenized_texts = torch.tensor(tokenized_texts)
attention_masks = torch.tensor(attention_masks)

In [16]:
# Set save model to Evalue loop
model.eval()
# Get model predict result
with torch.no_grad():
        outputs = model(tokenized_texts, token_type_ids=None,
        attention_mask=None,)
        # For eval mode, the first result of outputs is logits
        logits = outputs[0] 

In [17]:
predict_results = logits.detach().cpu().numpy()
result_arrays_soft = softmax(predict_results[0])
result_list = np.argmax(result_arrays_soft,axis=-1)

In [18]:
result = [tag2name[t] for t in result_list]
pretok_sent = ""
pretags = ""
for i, tok in enumerate(temp_token):
     if tok.startswith("##"):
         pretok_sent += tok[2:]
     else:
         pretok_sent += " " + tok
         pretags += " " + result[i]
pretok_sent = pretok_sent[1:]
pretags = pretags[1:]

s = pretok_sent.split()
t = pretags.split()

In [19]:
for i, mark in enumerate(attention_masks[0]):
    if mark>0:
        print("Token:%s"%(temp_token[i]))
#         print("Tag:%s"%(result_list[i]))
        print("Predict_Tag:%s"%(tag2name[result_list[i]]))
        #print("Posibility:%f"%(result_array[i][result_list[i]]))
        print()
        

Token:[CLS]
Predict_Tag:[CLS]

Token:Over
Predict_Tag:O

Token:the
Predict_Tag:O

Token:next
Predict_Tag:O

Token:three
Predict_Tag:O

Token:days
Predict_Tag:O

Token:he
Predict_Tag:O

Token:increased
Predict_Tag:O

Token:his
Predict_Tag:O

Token:activity
Predict_Tag:O

Token:gradually
Predict_Tag:O

Token:,
Predict_Tag:O

Token:was
Predict_Tag:O

Token:able
Predict_Tag:O

Token:to
Predict_Tag:O

Token:do
Predict_Tag:O

Token:stairs
Predict_Tag:O

Token:with
Predict_Tag:O

Token:Physical
Predict_Tag:B-treatment

Token:Therapy
Predict_Tag:I-treatment

Token:and
Predict_Tag:O

Token:had
Predict_Tag:O

Token:pain
Predict_Tag:B-problem

Token:which
Predict_Tag:O

Token:could
Predict_Tag:O

Token:be
Predict_Tag:O

Token:controlled
Predict_Tag:O

Token:with
Predict_Tag:O

Token:oral
Predict_Tag:B-treatment

Token:anal
Predict_Tag:I-treatment

Token:##ges
Predict_Tag:X

Token:##ics
Predict_Tag:X

Token:.
Predict_Tag:O

Token:[SEP]
Predict_Tag:[SEP]



In [24]:
text = ' '.join(temp_token)
re = ' '.join(result)

In [31]:
import docx
from docx.enum.text import WD_COLOR_INDEX
# Create an instance of a word document
doc = docx.Document()

# Add a Title to the document 
doc.add_heading('Results', 0)

# Creating paragraph with some content
para = doc.add_paragraph(''' ''')
  
flag_treatment, flag_problem, flag_test = 0, 0, 0 
for i in range(1, len(t)-1):
    if t[i] == 'B-treatment':
        flag_treatment = 1
        para.add_run(s[i]+' ').font.highlight_color = WD_COLOR_INDEX.RED
        # print(print_treatment(s[i]), end=' ')
    elif (t[i] == 'I-treatment' or t[i] == 'X') and flag_treatment == 1 :
        para.add_run(s[i]+' ').font.highlight_color = WD_COLOR_INDEX.RED
    elif t[i] == 'B-test':
        flag_test = 1
        para.add_run(s[i]+' ').font.highlight_color = WD_COLOR_INDEX.PINK
    elif (t[i] == 'I-test' or t[i] == 'X') and flag_test == 1 :
        para.add_run(s[i]+' ').font.highlight_color = WD_COLOR_INDEX.PINK
    elif t[i] == 'B-problem':
        flag_problem = 1
        para.add_run(s[i]+' ').font.highlight_color = WD_COLOR_INDEX.TURQUOISE
    elif (t[i] == 'I-problem' or t[i] == 'X') and flag_problem == 1 :
        para.add_run(s[i]+' ').font.highlight_color = WD_COLOR_INDEX.TURQUOISE    
    elif t[i] == 'O':
        flag_treatment, flag_problem, flag_test = 0, 0, 0 
        para.add_run(s[i]+' ').font.highlight_color = WD_COLOR_INDEX.AUTO
        

# # Adding more content to paragraph and highlighting them
# para.add_run(''' It contains well written, well thought and well-explained '''
#             ).font.highlight_color = WD_COLOR_INDEX.YELLOW
  
# # Adding more content to paragraph
# para.add_run('''computer science and programming articles, quizzes etc.''')
  
# Now save the document to a location 
doc.save('result.docx')


In [134]:
from termcolor import colored
def print_treatment(word):
    return colored(word,'white','on_red', attrs=['underline'])
def print_test(word):
    return colored(word, 'white','on_magenta')
def print_problem(word):
    return colored(word, 'magenta','on_cyan', attrs=['reverse', 'bold'])


In [135]:
print(print_problem('problem'))
print(print_treatment('treatment'))
print(print_test('test'))

problem
treatment
test


In [137]:
from termcolor import colored
location = {}
flag_treatment, flag_problem, flag_test = 0, 0, 0 
for i in range(1, len(t)-1):
    if t[i] == 'B-treatment':
        flag_treatment = 1
        print(print_treatment(s[i]), end=' ')
    elif (t[i] == 'I-treatment' or t[i] == 'X') and flag_treatment == 1 :
        print(print_treatment(s[i]), end=' ')
    elif t[i] == 'B-test':
        flag_test = 1
        print(print_test(s[i]), end=' ')
    elif (t[i] == 'I-test' or t[i] == 'X') and flag_test == 1 :
        print(print_test(s[i]), end=' ')
    elif t[i] == 'B-problem':
        flag_problem = 1
        print(print_problem(s[i]), end=' ')
    elif (t[i] == 'I-problem' or t[i] == 'X') and flag_problem == 1 :
        print(print_problem(s[i]), end=' ')    
    elif t[i] == 'O':
        flag_treatment, flag_problem, flag_test = 0, 0, 0 
        print(s[i], end=' ')

Over the next three days he increased his activity gradually , was able to do stairs with Physical Therapy and had pain which could be controlled with oral analgesics . 

In [ ]:
temp_lable = []
temp_token = []

# Add [CLS] at the front 
temp_lable.append('[CLS]')
temp_token.append('[CLS]')

# Tokenize words
# [lidocaine patch] -> lid ##oc ##aine patch
# [B-treatment I-treatment] -> B-treatment X X I-treatment
for word, lab in zip(word_list, label):
    token_list = tokenizer.tokenize(word)
    for m, token in enumerate(token_list):
        temp_token.append(token)
        if m == 0:
            temp_lable.append(lab)
        else:
            temp_lable.append('X')  
            
# Add [SEP] at the end
temp_lable.append('[SEP]')
temp_token.append('[SEP]')

In [5]:
class SentenceGetter(object) :
    """
    input a dataframe
    
    
    Generate sets of words and tags.
    self.sentence:
    Each sentence is a list
    [('Supraventricular', 'B-problem'),
    ('tachycardia', 'I-problem'),
    ('(', 'O'),
    ('on', 'O'),
    ('a', 'B-treatment'),
    ('beta', 'I-treatment'),
    ('blocker', 'I-treatment'),
    (')', 'O')]
    
    """
    def __init__(self, data):
        self.n_sent = 1
        self.data = data
        self.empty = False
        agg_func = lambda s: [(w, t) for w, t in zip(s["word"].values.tolist(),
                                                        #    s["POS"].values.tolist(),
                                                           s["tag"].values.tolist())]
        self.grouped = self.data.groupby("sentence #").apply(agg_func)
        self.sentences = [s for s in self.grouped]
    
    def get_next(self):
        try:
            s = self.grouped["sentence: {}".format(self.n_sent)]
            self.n_sent += 1
            return s
        except:
            return None
    
    def get_examples(self):
        return random.sample(self.sentences, 10)
    
    def get_sentences(self):
        return[[s[0] for s in sent] for sent in self.sentences]
        
    def get_labels(self):
        return [[s[1] for s in sent] for sent in self.sentences]

In [6]:
class InputGenerater(object):
    def __init__(self, sentences, labels) -> None:
        self.tokenized_texts = []
        self.word_piece_labels = []
        i_inc = 0
        for word_list,label in (zip(sentences,labels)):
            temp_lable = []
            temp_token = []
            
            # Add [CLS] at the front 
            temp_lable.append('[CLS]')
            temp_token.append('[CLS]')
            
            for word,lab in zip(word_list,label):
                token_list = tokenizer.tokenize(word)
                for m,token in enumerate(token_list):
                    temp_token.append(token)
                    if m==0:
                        temp_lable.append(lab)
                    else:
                        temp_lable.append('X')  
                        
            # Add [SEP] at the end
            temp_lable.append('[SEP]')
            temp_token.append('[SEP]')
            
            self.tokenized_texts.append(temp_token)
            self.word_piece_labels.append(temp_lable)
            
            if 5 > i_inc:
                print("No.%d,len:%d"%(i_inc,len(temp_token)))
                print("texts:%s"%(" ".join(temp_token)))
                print("No.%d,len:%d"%(i_inc,len(temp_lable)))
                print("lables:%s"%(" ".join(temp_lable)))
            i_inc +=1
            
        self.input_ids = pad_sequences([tokenizer.convert_tokens_to_ids(txt) for txt in self.tokenized_texts],
                    maxlen=config.MAX_LEN, dtype="long", truncating="post", padding="post")
            
    def get_input_ids(self):    
        return self.input_ids
    
    def get_tags(self):
        tags = pad_sequences([[get_tag2idx().get(l) for l in lab] for lab in self.word_piece_labels],
                        maxlen=config.MAX_LEN, value=get_tag2idx()["O"], padding="post",
                        dtype="long", truncating="post")
        return tags
    def get_attention_masks(self):
        attention_masks = [[int(i>0) for i in ii] for ii in self.input_ids]
        return attention_masks
    def get_segment_ids(self):
        segment_ids = [[0] * len(input_id) for input_id in self.input_ids]
        return segment_ids


def convert_to_tensor(*inputs, drop_last=False):
    data = TensorDataset(*tuple(torch.tensor(inputs)))
    data_sampler = RandomSampler(data)
    # Drop last can make batch training better for the last one
    dataloader = DataLoader(data, sampler=data_sampler, batch_size=config.BATCH_NUM, drop_last=drop_last)
    return dataloader

In [10]:
df_test = pd.read_csv(config.data_path_test, sep="\t").astype(str)
sg = SentenceGetter(df_test)
sentences = sg.get_sentences()
tags = sg.get_labels()
test_sets = InputGenerater(sentences=sentences, labels=tags)

test_inputs = test_sets.get_input_ids()
test_tags = test_sets.get_tags()
test_attetion_masks = test_sets.get_attention_masks()

test_dataloader = utils.convert_to_tensor(test_inputs, test_attetion_masks, test_tags)


No.0,len:5
texts:[CLS] admission date : [SEP]
No.0,len:5
lables:[CLS] O O O [SEP]
No.1,len:7
texts:[CLS] 2014 - 12 - 29 [SEP]
No.1,len:7
lables:[CLS] O X X X X [SEP]
No.2,len:6
texts:[CLS] all ##er ##gies : [SEP]
No.2,len:6
lables:[CLS] O X X O [SEP]
No.3,len:4
texts:[CLS] 17 units [SEP]
No.3,len:4
lables:[CLS] O O [SEP]
No.4,len:22
texts:[CLS] includes a history of at ##rial fi ##bri ##lla ##tion with good heart rate control on dig ##ox ##in . [SEP]
No.4,len:22
lables:[CLS] O O O O B-problem X I-problem X X X O O B-treatment I-treatment I-treatment O B-treatment X X O [SEP]


NameError: name 'pad_sequences' is not defined

In [40]:
import glob

In [43]:
f = glob.glob('processed/test/*')

In [45]:
os.listdir(config.INDIVIDUAL_TEST)

['0289.tsv',
 '0090.tsv',
 '0246.tsv',
 '0390.tsv',
 '0365.tsv',
 '0174.tsv',
 '0282.tsv',
 '0305.tsv',
 '0150.tsv',
 '0101.tsv',
 '0086.tsv',
 '0357.tsv',
 '0230.tsv',
 '0294.tsv',
 '0377.tsv',
 '0266.tsv',
 '0050.tsv',
 '0026.tsv',
 '0049.tsv',
 '0029.tsv',
 '0005.tsv',
 '0081.tsv',
 '0466.tsv',
 '0245.tsv',
 '0378.tsv',
 '0463.tsv',
 '0445.tsv',
 '0053.tsv',
 '0477.tsv',
 '0461.tsv',
 '0362.tsv',
 '0473.tsv',
 '0309.tsv',
 '0415.tsv',
 '0222.tsv',
 '0329.tsv',
 '0474.tsv',
 '0393.tsv',
 '0345.tsv',
 '0109.tsv',
 '0129.tsv',
 '0454.tsv',
 '0366.tsv',
 '0082.tsv',
 '0439.tsv',
 '0173.tsv',
 '0094.tsv',
 '0025.tsv',
 '0270.tsv',
 '0133.tsv',
 '0261.tsv',
 '0153.tsv',
 '0237.tsv',
 '0074.tsv',
 '0046.tsv',
 '0066.tsv',
 '0037.tsv',
 '0338.tsv',
 '0285.tsv',
 '0425.tsv',
 '0322.tsv',
 '0385.tsv',
 '0446.tsv',
 '0141.tsv',
 '0138.tsv',
 '0121.tsv',
 '0421.tsv',
 '0389.tsv',
 '0186.tsv',
 '0233.tsv',
 '0134.tsv',
 '0054.tsv',
 '0013.tsv',
 '0190.tsv',
 '0185.tsv',
 '0409.tsv',
 '0205.tsv',

In [61]:
f = open("eval_results.txt", "r")
print(f.read())

f1 socre:
1.0

Accuracy score:
1.0

              precision    recall  f1-score   support

     problem     1.0000    1.0000    1.0000         1
        test     1.0000    1.0000    1.0000         1
   treatment     1.0000    1.0000    1.0000         4

   micro avg     1.0000    1.0000    1.0000         6
   macro avg     1.0000    1.0000    1.0000         6
weighted avg     1.0000    1.0000    1.0000         6

